In [1]:
!pip install fastapi uvicorn
!pip install langchain langchain-community
!pip install chromadb
!pip install transformers
!pip install accelerate
!pip install huggingface_hub
!pip install langchain-google-genai
!pip install google-generativeai
!pip install html2text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23
     ━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.6 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.17
    Uninstalling google-ai-generativelanguage-0.6.17:
      Successfully uninstalled google-ai-generativelanguage-0.6.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.3 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [3]:
import requests
import json
import os
from google.colab import userdata

In [5]:
# Notion API 인증 설정
# 실제 사용시 Colab의 보안 탭에서 시크릿을 설정하세요
NOTION_TOKEN = userdata.get('NOTION_TOKEN')  # Colab에서 안전하게 토큰 관리
NOTION_VERSION = "2022-06-28"  # API 버전

headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": NOTION_VERSION,
    "Content-Type": "application/json"
}

def get_page_content(page_id):
    """
    페이지 ID를 기반으로 페이지 내용을 가져옵니다.

    Args:
        page_id (str): Notion 페이지 ID (대시 제외)

    Returns:
        dict: 페이지 속성 및 내용
    """
    # 페이지 속성 가져오기
    page_url = f"https://api.notion.com/v1/pages/{page_id}"
    response = requests.get(page_url, headers=headers)

    if response.status_code != 200:
        print(f"오류 발생: {response.status_code}")
        print(response.text)
        return None

    page_data = response.json()

    # 페이지 내용(블록) 가져오기
    blocks_url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return page_data

    blocks_data = response.json()

    # 페이지 데이터에 블록 내용 추가
    page_data["content"] = blocks_data["results"]

    return page_data

def fetch_all_blocks(block_id):
    """
    블록의 모든 하위 블록을 재귀적으로 가져옵니다.

    Args:
        block_id (str): 블록 ID

    Returns:
        list: 모든 하위 블록 목록
    """
    blocks_url = f"https://api.notion.com/v1/blocks/{block_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return []

    blocks_data = response.json()["results"]

    # 하위 블록이 있는 블록 타입들
    has_children_types = [
        "paragraph", "bulleted_list_item", "numbered_list_item",
        "toggle", "child_page", "child_database", "column_list",
        "column", "table", "synced_block"
    ]

    all_blocks = []

    for block in blocks_data:
        all_blocks.append(block)

        # 하위 블록이 있는 경우 재귀적으로 가져오기
        if block.get("has_children") and block.get("type") in has_children_types:
            children = fetch_all_blocks(block["id"])
            # 하위 블록이 있는 경우에만 children 키 추가
            if children:
                block["children"] = children

    return all_blocks

def extract_text_content(blocks):
    """
    블록에서 텍스트 내용만 추출합니다.

    Args:
        blocks (list): 블록 목록

    Returns:
        str: 추출된 텍스트 내용
    """
    text_content = ""

    for block in blocks:
        block_type = block.get("type")

        if block_type == "paragraph":
            rich_text = block.get("paragraph", {}).get("rich_text", [])
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_1":
            rich_text = block.get("heading_1", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "# " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_2":
            rich_text = block.get("heading_2", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "## " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_3":
            rich_text = block.get("heading_3", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "### " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "bulleted_list_item":
            rich_text = block.get("bulleted_list_item", {}).get("rich_text", [])
            text_content += "• "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "numbered_list_item":
            rich_text = block.get("numbered_list_item", {}).get("rich_text", [])
            # 실제로는 번호가 순차적으로 증가해야 하지만, 여기서는 간단하게 표시
            text_content += "1. "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "to_do":
            rich_text = block.get("to_do", {}).get("rich_text", [])
            checked = block.get("to_do", {}).get("checked", False)
            text_content += "[{}] ".format("x" if checked else " ")
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "code":
            rich_text = block.get("code", {}).get("rich_text", [])
            language = block.get("code", {}).get("language", "")
            text_content += f"```{language}\n"
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n```\n\n"

        # 하위 블록이 있는 경우 재귀적으로 처리
        if "children" in block:
            text_content += extract_text_content(block["children"])

    return text_content

# 메인 실행 코드
def main():
    # 사용자로부터 페이지 ID 입력받기
    page_id = input("Notion 페이지 ID를 입력하세요 (예: 053b5fdc70c34fe8b931c06e320c3083): ")
    page_id = page_id.replace("-", "")  # 대시가 있으면 제거

    print("페이지 내용을 가져오는 중...")

    # 페이지 내용 가져오기
    page_data = get_page_content(page_id)

    if not page_data:
        print("페이지를 가져올 수 없습니다.")
        return

    # 모든 블록 가져오기 (재귀적으로)
    print("모든 블록을 재귀적으로 가져오는 중...")
    all_blocks = fetch_all_blocks(page_id)

    # 텍스트 내용 추출
    text_content = extract_text_content(all_blocks)

    # 결과 저장
    print("\n=== 페이지 기본 정보 ===")
    page_title = page_data.get("properties", {}).get("title", {}).get("title", [])
    if page_title:
        title_text = " ".join([text.get("plain_text", "") for text in page_title])
        print(f"제목: {title_text}")

    print("\n=== 페이지 내용 ===")
    print(text_content)

    # JSON 형식으로 저장
    with open("notion_page_content.json", "w", encoding="utf-8") as f:
        json.dump(page_data, f, ensure_ascii=False, indent=2)

    # 텍스트 형식으로 저장
    with open("notion_page_content.txt", "w", encoding="utf-8") as f:
        f.write(text_content)

    print("\n페이지 내용이 'notion_page_content.json'과 'notion_page_content.txt'로 저장되었습니다.")

In [6]:
if __name__ == "__main__":
    main()

Notion 페이지 ID를 입력하세요 (예: 053b5fdc70c34fe8b931c06e320c3083): 1d7f6e2d24c480dda518e47a53509b63
페이지 내용을 가져오는 중...
모든 블록을 재귀적으로 가져오는 중...

=== 페이지 기본 정보 ===
제목: Retrieval-Augmented Generation for Large Language Models: A Survey

=== 페이지 내용 ===
# 1. Introduction

### RAG 기술 발전 Taxonomy

### 1단계: 초기 등장: Transformer

• PTM + 외부 지식 → 성능 향상
• 사전학습 기술 정교화
### 2단계: 전환점: ChatGPT 등장

• LLM의 ICL(In-Context Learning) 능력 강화
• RAG → 복잡한 질의에 정확한 정보 제공
### 3단계: 고도화 연구

• 단순 검색 보조 → LLM 파인튜닝과 통합
• RAG + LLM 구조적 결합


⇒ 본 논문에서는 RAG의 발전 과정을 체계적으로 정리하고, LLM과의 통합을 중심으로 기술 패러다임과 연구 방법론을 분석한다



# 2. Overview of RAG

질문에 답변하는 RAG의 예시: 대개 세 단계로 구성된다. 

1) Indexing. Document가 chunk로 쪼개진다. 

2) Retrieval. semantic similarity를 기반으로 질문과 가장 관련 있는 Top_k개의 chunk들을 획득한다. 

3) Generation. 원래 question과 획득한 chunk를 LLM에 넣어서 최종 결과를 생성한다.



## Naive RAG

• Process: 간단한 "Retrieve-Read" 파이프라인:
1. Indexing:
• 데이터 정리 및 분할(PDF, HTML 등 → plain text).
• 분할된 텍스트를 vector embeddings으로 변환하여 vector database에 저장.
1. Retrieval:
• 사용자 쿼리를 벡